In [1]:
# ok. You have an idea of what is PCA - finding dimensions, explaining the data the best way
# how to run PCA?

import numpy as np
from sklearn.decomposition import PCA

# print 3 symbols in fixed notation
np.set_printoptions(precision=3, suppress=True)

X = np.matrix([
    [-1, 0.1, 1],
    [-3, 0.01, 3],
    [-10, 0.02, 10],
    [8, -0.01, -8],
    [6, -0.3, -6],
    [0, -0, 0],
])

pca = PCA(n_components=1)
X_reduced = pca.fit_transform(X)
X_reduced2 = PCA(n_components=2).fit_transform(X)

# NB these are just distances from 0!
print("1 component")
print(X_reduced)

# NB second dimension captures remaining noise
# It is much smaller in absolute values.
print("2 components")
print(X_reduced2)

1 component
[[  1.415]
 [  4.243]
 [ 14.142]
 [-11.313]
 [ -8.487]
 [  0.   ]]
2 components
[[  1.415  -0.119]
 [  4.243  -0.008]
 [ 14.142   0.055]
 [-11.313  -0.104]
 [ -8.487   0.207]
 [  0.     -0.03 ]]


In [2]:
# how to obtain transformation matrix T from 3D to 1D or 2D?
# Obviuosly, solve an equation!
#     (X - μ) * T = X_reduced
# where (X - μ) is just a "centered" matrix X.
# 
# This is a basic matrix equation. If (X - μ) is square, you can just
# write T = (X - μ)^-1 * X_reduced.
# For non-square matrix you can go with approximation of T.
# Best well-known method is "least square error" method

μ = np.mean(X, 0)
X_ = X - μ
# use LSE method
T1 = np.linalg.pinv(X_.T * X_) * X_.T * X_reduced
T2 = np.linalg.pinv(X_.T * X_) * X_.T * X_reduced2

# Computed values are exactly the same values as PCA provided.
# That means we solved the problem correctly.
print("Transformation matrix 1D:")
print(T1)
print("Transformed matrix X 1D:")
print(((X - μ) * T1))

print("Transformation matrix 2D:")
print(T2)
print("Transformed matrix X 2D:")
print(((X - μ) * T2))

# But we can also project other points of bigger dimensions into smaller dimensions.
print("Apply trasformation to NEW  unseen data:")
print(np.matrix([[-50, 1, 50]]) * T1)

Transformation matrix 1D:
[[-0.707]
 [ 0.007]
 [ 0.707]]
Transformed matrix 1D:
[[  1.415]
 [  4.243]
 [ 14.142]
 [-11.313]
 [ -8.487]
 [  0.   ]]
Transformation matrix 2D:
[[-0.707 -0.005]
 [ 0.007 -1.   ]
 [ 0.707  0.005]]
Transformed matrix 2D:
[[  1.415  -0.119]
 [  4.243  -0.008]
 [ 14.142   0.055]
 [-11.313  -0.104]
 [ -8.487   0.207]
 [  0.     -0.03 ]]
Apply trasformation to NEW data:
[[70.716]]


In [5]:
# what about SVD?
# SVD breaks a matrix into 3. It allows us to manipulate approximation of 
# initial matrix by removing rows from matrices of decomposition

from numpy.linalg import svd

# X - μ = exactly equals = U * Σ * V_T
# in or example X is 6x3. Thus, U is 6x6, Σ is 6x3 and V_T is 3x3.

U, s, V_T = svd(X - μ)
print("==================== U ======================")
print(U)
print("==================== Σ ======================")
# place values of "s" into diagonal of Σ
Σ = np.zeros((U.shape[1], V_T.shape[0]), dtype=float)
Σ[:V_T.shape[0], :V_T.shape[0]] = np.diag(s)
print(Σ)
print("==================== V ======================")
print(V_T)

# ok. Let  X - μ = (UΣ)*V_T
# where UΣ is 6x3 and V_T is 3x3
# to reduce dimensions we remove middle numbers:
# let UΣ is 6x1 and V_T is 1x3

UΣ = U * Σ
print("Look, we got the same matrix 1D (UΣ = PCA(X))!")
print(UΣ[:,:1])
print("Look, we got the same matrix 2D (UΣ = PCA(X))!")
print(UΣ[:,:2])

# restore original? - no problem.
# X ~ UΣ[-cols] * V_T[-rows] + μ

print("X approximation with SVD. 1D")
print(UΣ[:,:1] * V_T[:1, :] + μ)
print("X approximation with SVD. 1D")
print(UΣ[:,:2] * V_T[:2, :] + μ)

# to sum up. Now we have matrix T to reduce dimesions and matrix V_T to "restore" original vector.
# Hmm... can we use this to generate texts?

==================== U ======================
[[-0.069 -0.445  0.874  0.134  0.123 -0.012]
 [-0.207 -0.031 -0.026  0.636 -0.732  0.123]
 [-0.69   0.206 -0.09   0.416  0.547 -0.02 ]
 [ 0.552 -0.389 -0.293  0.572  0.357 -0.066]
 [ 0.414  0.771  0.373  0.271  0.091  0.113]
 [-0.    -0.112 -0.051 -0.063  0.118  0.983]]
==================== Σ ======================
[[20.494  0.     0.   ]
 [ 0.     0.268  0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]
==================== V ======================
[[ 0.707 -0.007 -0.707]
 [-0.005 -1.     0.005]
 [ 0.707 -0.     0.707]]
Look, we got the same matrix 1D (UΣ = PCA(X))!
[[ -1.415]
 [ -4.243]
 [-14.142]
 [ 11.313]
 [  8.487]
 [ -0.   ]]
Look, we got the same matrix 2D (UΣ = PCA(X))!
[[ -1.415  -0.119]
 [ -4.243  -0.008]
 [-14.142   0.055]
 [ 11.313  -0.104]
 [  8.487   0.207]
 [ -0.     -0.03 ]]
X approximation with SVD. 1D
[[ -1.001  -0.019   1.001]
 [ -3.      0.002   3.   ]
 [-10.      0.